In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import scipy.stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("train.csv")
print (train.shape)
train.head()

(614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
test = pd.read_csv("test.csv")
print (test.shape)
test.head()

(367, 12)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [4]:
train[["Gender","Married","Dependents","CoapplicantIncome","LoanAmount"]][train.Married == "Yes"]

,Gender,Married,Dependents,CoapplicantIncome,LoanAmount
1,Male,Yes,1,1508.0,128.0
2,Male,Yes,0,0.0,66.0
3,Male,Yes,0,2358.0,120.0
5,Male,Yes,2,4196.0,267.0
6,Male,Yes,0,1516.0,95.0
7,Male,Yes,3+,2504.0,158.0
8,Male,Yes,2,1526.0,168.0
9,Male,Yes,1,10968.0,349.0
10,Male,Yes,2,700.0,70.0
11,Male,Yes,2,1840.0,109.0


In [5]:
#try to predict oder of age of person based on his martial status and no of dependents
# dataset = [train,test]
# for i in dataset:
#     l = []
#     for j in i.Married.index:
#         m =(i.Married[j] == "Yes")
#         if(i.Dependents[j] == "0"):
#             n = 1
#         if (i.Dependents[j] == "1"):
#             n = 2
#         if (i.Dependents[j] == "2"):
#             n = 3
#         if (i.Dependents[j] == "3+"):
#             n = 4
#         l.append(m*n)
#     i["age"] = l
## but this feature decresed the accuracy or has no feature_importance in xgboost

In [6]:
##E = P×r×(1 + r)n/((1 + r)n - 1)
##average rate of intrest =8.6(for women) 8.65(for men)
##r = 8.6/(12*100) = 0.007166,8.65/(12*100) =0.007208
#E = P*r*(1 + r)**n/((1 + r)**n - 1)
dataset = [train,test]
for i in dataset:
    l = []
    for j in i.Gender.index:
        if i.Gender[j] == "Male":
            r = 8.65/(12*100)
        else:
            r = 8.6/(12*100)
        P = i.LoanAmount[j]*1000
        n = i.Loan_Amount_Term[j]
        E = P*r*(1 + r)**n/((1 + r)**n - 1)
        l.append(E)
    i["EMI"] = l

In [7]:
dataset = [train,test]
for i in dataset:
    i["income"] = i["ApplicantIncome"] + i["CoapplicantIncome"]
#     i["app_income/loan"] = [x[0]/x[1] for x in zip(i["ApplicantIncome"],i["LoanAmount"])]
    i["income/loan"] = [x[0]/x[1] for x in zip(i["income"],i["LoanAmount"])]
    i.drop(["CoapplicantIncome"],axis = 1,inplace = True)
#     i["remain"] = i["income"] - i["EMI"]
#     i["remain2"] = i["ApplicantIncome"] - i["EMI"]

## seen corr in scatter but overall accuracy decreases
#     i["emi/income"] = [x[0]/x[1] for x in zip(i["EMI"],i["income"])]
#     i["emi/app_income"] = [x[0]/x[1] for x in zip(i["EMI"],i["ApplicantIncome"])]

In [8]:
train.shape,test.shape

((614, 15), (367, 14))

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 15 columns):
Loan_ID             614 non-null object
Gender              601 non-null object
Married             611 non-null object
Dependents          599 non-null object
Education           614 non-null object
Self_Employed       582 non-null object
ApplicantIncome     614 non-null int64
LoanAmount          592 non-null float64
Loan_Amount_Term    600 non-null float64
Credit_History      564 non-null float64
Property_Area       614 non-null object
Loan_Status         614 non-null object
EMI                 578 non-null float64
income              614 non-null float64
income/loan         592 non-null float64
dtypes: float64(6), int64(1), object(8)
memory usage: 72.0+ KB


In [10]:
for i in train.columns[train.dtypes == "object"]:
    train[i].fillna(train[i].mode()[0],inplace = True)
for i in train.columns[train.dtypes != "object"]:
    train[i].fillna(train[i].mean(),inplace = True)
for i in test.columns[test.dtypes == "object"]:
    test[i].fillna(train[i].mode()[0],inplace = True)
for i in train.columns[train.dtypes != "object"]:
    test[i].fillna(test[i].mean(),inplace = True)

    
## tried to predictive missing value input but online accuracy decreased while doing so.
# test.Loan_Status = np.nan
# alldata = pd.concat([train,test],axis = 0)

# # all_col = list(set(alldata.columns) - set(["Loan_ID"]))
# # alldata["total_income"] = alldata["ApplicantIncome"] + alldata["CoapplicantIncome"]
# # alldata.drop(["Loan_ID"],inplace = True,axis = 1)  ##"ApplicantIncome","CoapplicantIncome",
# # alldata.Dependents[alldata.Dependents == '3+'] = 3
# # alldata.Dependents = alldata.Dependents.astype('float64')
# print (alldata.shape)

# ##for married 
# alldata.Married[alldata.Married.isnull()] = alldata.Married.unique()[1]

# #filling the missing values of gender
# alldata.Gender[alldata.Gender.isnull()] = np.random.choice(["Male","Female"],size = sum(alldata.Gender.isnull()),replace  = True)

# #filling the missing values of self_employed
# alldata.Self_Employed[alldata.Self_Employed.isnull()] =np.random.choice(['Yes','No'],size = sum(alldata.Self_Employed.isnull()))

# def input_missing_value(df_,col):
#     df_loan_id = df_.Loan_ID
#     df_.drop(["Loan_ID"],inplace = True,axis = 1)
#     x = list(df_.columns[~df_.isnull().any()])
#     x.append(col)
#     df = df_[x]
#     train = df[df[col].notnull()]
#     test = df[df[col].isnull()]
    
#     le = LabelEncoder()
#     for x in train.columns[train.dtypes == 'object']:
#         train[x] = le.fit_transform(train[x])
#         test[x][test[x].notnull()]  = le.transform(test[x][test[x].notnull()])
#         test[x] = test[x].astype('float') 

#     target = train[col]
#     train.drop([col],axis = 1,inplace = True)
#     test.drop([col],axis = 1,inplace = True)
# #     from sklearn.neighbors import KNeighborsClassifier
# #     cl = KNeighborsClassifier(n_neighbors=1)
#     from xgboost import XGBClassifier
#     cl = XGBClassifier()
#     cl.fit(train,target)
#     pred = cl.predict(test)
#     if(col in df.columns[df.dtypes == 'object']):
#         le.fit_transform(df[col][df[col].notnull()])
#         pred = le.inverse_transform(pred)
#     df_[col][df_[col].isnull()] = pred
#     df_["Loan_ID"] =df_loan_id 
#     return df_
        

In [11]:
# alldata = input_missing_value(alldata,'Loan_Amount_Term')
# alldata= input_missing_value(alldata,'Dependents')
# alldata= input_missing_value(alldata,'LoanAmount')
# alldata= input_missing_value(alldata,'Credit_History')
# alldata.dtypes

In [12]:
# def input_missing_value(df_,col):
#     df_loan_id = df_.Loan_ID
#     df_.drop(["Loan_ID"],inplace = True,axis = 1)
#     x = list(df_.columns[~df_.isnull().any()])
#     x.append(col)
#     df = df_[x]
#     train = df[df[col].notnull()]
#     test = df[df[col].isnull()]
    
#     le = LabelEncoder()
#     for x in train.columns[train.dtypes == 'object']:
#         train[x] = le.fit_transform(train[x])
#         test[x][test[x].notnull()]  = le.transform(test[x][test[x].notnull()])
#         test[x] = test[x].astype('float') 


#     target = train[col]
#     train.drop([col],axis = 1,inplace = True)
#     test.drop([col],axis = 1,inplace = True)
# #     from sklearn.neighbors import KNeighborsRegressor
# #     cl = KNeighborsRegressor(n_neighbors=1)
# #     
#     from xgboost import XGBRegressor
#     cl = XGBRegressor()
#     cl.fit(train,target)
#     pred = cl.predict(test)
#     if(col in df.columns[df.dtypes == 'object']):
#         le.fit_transform(df[col][df[col].notnull()])
#         pred = le.inverse_transform(pred)
#     df_[col][df_[col].isnull()] = pred
#     df_["Loan_ID"] =df_loan_id 
#     return df_

In [13]:
# alldata= input_missing_value(alldata,'EMI')
# alldata = input_missing_value(alldata,"income/loan")
# alldata.info()

In [14]:
# train = alldata[alldata.Loan_Status.notnull()]
# # target = pd.DataFrame(train.Loan_Status,columns =["Loan_Status"])
# # target[target == "Y"] = 1
# # target[target=="N"] = 0
# # target = target.astype("int")
# # train.drop(["Loan_Status"],axis = 1,inplace = True)
# test = alldata[alldata.Loan_Status.isnull()]
# test.drop(["Loan_Status"],axis = 1,inplace = True)
# print (train.shape,test.shape)


In [15]:
for i in list(set(train.columns[train.dtypes == "object"]) -set(["Loan_ID",'Loan_Status'])):
    le = LabelEncoder()
    train[i] = le.fit_transform(train[i].astype("str"))
    test[i] = le.transform(test[i].astype("str"))

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 15 columns):
Loan_ID             614 non-null object
Gender              614 non-null int64
Married             614 non-null int64
Dependents          614 non-null int64
Education           614 non-null int64
Self_Employed       614 non-null int64
ApplicantIncome     614 non-null int64
LoanAmount          614 non-null float64
Loan_Amount_Term    614 non-null float64
Credit_History      614 non-null float64
Property_Area       614 non-null int64
Loan_Status         614 non-null object
EMI                 614 non-null float64
income              614 non-null float64
income/loan         614 non-null float64
dtypes: float64(6), int64(7), object(2)
memory usage: 72.0+ KB


In [17]:
train.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,EMI,income,income/loan
0,LP001002,1,0,0,0,0,5849,146.412162,360.0,1.0,2,Y,1208.877399,5849.0,51.225813
1,LP001003,1,1,1,0,0,4583,128.000000,360.0,1.0,0,N,997.848496,6091.0,47.585938
2,LP001005,1,1,0,0,1,3000,66.000000,360.0,1.0,2,Y,514.515631,3000.0,45.454545
3,LP001006,1,1,0,1,0,2583,120.000000,360.0,1.0,2,Y,935.482965,4941.0,41.175000
4,LP001008,1,0,0,0,0,6000,141.000000,360.0,1.0,2,Y,1099.192483,6000.0,42.553191
5,LP001011,1,1,2,0,1,5417,267.000000,360.0,1.0,2,Y,2081.449596,9613.0,36.003745
6,LP001013,1,1,0,1,0,2333,95.000000,360.0,1.0,2,Y,740.590680,3849.0,40.515789
7,LP001014,1,1,3,0,0,3036,158.000000,360.0,0.0,1,N,1231.719237,5540.0,35.063291
8,LP001018,1,1,2,0,0,4006,168.000000,360.0,1.0,2,Y,1309.676150,5532.0,32.928571
9,LP001020,1,1,1,0,0,12841,349.000000,360.0,1.0,1,N,2720.696289,23809.0,68.220630


In [18]:
test.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,EMI,income,income/loan
0,LP001015,1,1,0,0,0,5720,110.0,360.0,1.000000,2,857.526051,5720,52.000000
1,LP001022,1,1,1,0,0,3076,126.0,360.0,1.000000,2,982.257113,4576,36.317460
2,LP001031,1,1,2,0,0,5000,208.0,360.0,1.000000,2,1621.503805,6800,32.692308
3,LP001035,1,1,2,0,0,2340,100.0,360.0,0.825444,2,779.569137,4886,48.860000
4,LP001051,1,0,0,1,0,3276,78.0,360.0,1.000000,2,608.063927,3276,42.000000
5,LP001054,1,1,0,1,1,2165,152.0,360.0,1.000000,2,1184.945089,5587,36.756579
6,LP001055,0,0,1,1,0,2226,59.0,360.0,1.000000,1,457.846912,2226,37.728814
7,LP001056,1,1,2,1,0,3881,147.0,360.0,0.000000,0,1145.966632,3881,26.401361
8,LP001059,1,1,2,0,0,13633,280.0,240.0,1.000000,2,2456.552747,13633,48.689286
9,LP001067,1,0,0,1,0,2400,123.0,360.0,1.000000,1,958.870039,4800,39.024390


In [19]:
tra_col = sorted(list(set (train.columns)- set(["Loan_ID","Loan_Status"])))
test_col = sorted(list(set (test.columns)- set(["Loan_ID"])))
print(tra_col,test_col)
# rf = RandomForestClassifier()
# rf.fit(train[tra_col],train["Loan_Status"])
# rf.predict(test[test_col])

import xgboost as xgb
from xgboost import XGBClassifier
model = XGBClassifier()

## tried to optimise the algo but accuracy decreased on solution checker

# from sklearn.model_selection import GridSearchCV
# def cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=3):
#     gs = GridSearchCV(clf, param_grid=parameters, cv=n_folds)
#     gs.fit(Xtrain, ytrain)
#     print ("BEST PARAMS", gs.best_params_)
#     best = gs.best_estimator_
#     return best

# # from sklearn.cross_validation import train_test_split
# def do_classify(clf, parameters, Xtrain, Xtest, ytrain,  standardize=True, train_size=0.66):
#     subdf=Xtrain
#     if standardize:
#         subdf=(subdf - subdf.mean())/subdf.std()  
#     Xtrain=subdf
#     ytrain=ytrain
#     subdf = Xtest
#     if standardize:
#         subdf=(subdf - subdf.mean())/subdf.std()
#     Xtest=subdf
    
# #     Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size,random_state = 0)
#     clf = cv_optimize(clf, parameters, Xtrain, ytrain)
#     clf=clf.fit(Xtrain, ytrain)
#     training_accuracy = clf.score(Xtrain, ytrain)
#     print ("Accuracy on training data: %0.2f" % (training_accuracy))
#     return clf.predict(Xtest)


# prediction= do_classify(model,{"colsample_bylevel":[.2,.3,.5],"n_estimators":[5,7,10]},train[tra_col],test[test_col],train["Loan_Status"])
# print(prediction)

model.fit(train[tra_col],train["Loan_Status"])
# from scipy import sparse
model.predict(test[test_col])

['ApplicantIncome', 'Credit_History', 'Dependents', 'EMI', 'Education', 'Gender', 'LoanAmount', 'Loan_Amount_Term', 'Married', 'Property_Area', 'Self_Employed', 'income', 'income/loan'] ['ApplicantIncome', 'Credit_History', 'Dependents', 'EMI', 'Education', 'Gender', 'LoanAmount', 'Loan_Amount_Term', 'Married', 'Property_Area', 'Self_Employed', 'income', 'income/loan']


array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N',
       'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [20]:
model.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)>

In [21]:
result = pd.DataFrame({"Loan_ID":(test.Loan_ID),"Loan_Status":model.predict(test[test_col])}).reset_index(drop = True)
# result['Loan_Status'] = result['Loan_Status'].map(lambda x: to_labels(x))
result[["Loan_ID","Loan_Status"]].to_csv("loan_prediction_baseline.csv",index = False)

In [22]:
sorted([(x[0],x[1]) for x in zip(model.feature_importances_,tra_col)])

[(0.0, 'Self_Employed'),
 (0.015202703, 'Married'),
 (0.018581081, 'Education'),
 (0.021959459, 'Dependents'),
 (0.030405406, 'Gender'),
 (0.048986487, 'Loan_Amount_Term'),
 (0.065878376, 'Property_Area'),
 (0.074324325, 'Credit_History'),
 (0.11317568, 'LoanAmount'),
 (0.12331081, 'EMI'),
 (0.14358108, 'income/loan'),
 (0.16554055, 'ApplicantIncome'),
 (0.17905405, 'income')]